In [1]:
"""
NOTE: ON RE-RUNNING THIS SCRIPT, THE OLD OHLC CSVS WILL BE OVERWRITTEN
PLEASE STORE THE OLD CSVS SOMEWHERE SAFE THEN YOU CAN PROCEED
"""

from fyers_apiv3.FyersWebsocket import data_ws
from fyers_apiv3 import fyersModel
from datetime import datetime,timedelta
import pandas as pd
import threading
from pytz import timezone
import time
import pandas as pd
from csv import DictWriter
import os

#generate trading session
client_id = open("../client_ID.txt",'r').read()
access_token = open("../access_token.txt",'r').read()

timeframe = 1   # timeframe in minutes

vol_data = {}

csv_data = {} 

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

timeframe_counter = 1

prev_vol = 0

def onmessage(message):

    global timeframe_counter
    global timeframe
    global prev_vol

    print("Response", message)
    ms = message['exch_feed_time']
    curr_time = datetime.fromtimestamp(ms)
    print("message exch_feed_time:", curr_time)

    if curr_time.second == 0:
        timeframe_counter += 1
    
    if timeframe_counter == timeframe:
        for symbol in vol_data:
            try:
                # find ohlc and save in csv
                curr_vol = message['vol_traded_today']

                #print(vol_data[symbol])
                csv_dict = {'minute': str(curr_time), 'symbol': str(symbol),
                            'total_volume': int(curr_vol), f'{timeframe}_min_vol': int(curr_vol - prev_vol)}
                
                print(curr_vol - prev_vol)
                prev_vol = curr_vol


                if csv_data.get(symbol) == None:
                    csv_data[symbol] = []

                csv_data[symbol].append(csv_dict)
                df_dictionary = pd.DataFrame(csv_data[symbol])
                df_dictionary.to_csv(f'{symbol.replace(":", "_")}.csv')

                #generate_csv(symbol.replace(":", "_"), ['symbol', 'high', 'low', 'open', 'close', 'minute'], csv_dict)

                print("CSV MADE")
                print(csv_dict)

            except Exception as e:
                print(e)
            
            vol_data[symbol] = []

        timeframe_counter = 0

    else:
        if vol_data.get(message['symbol']) == None:
            vol_data[message['symbol']] = []
        
        vol_data[message['symbol']].append(float(message['vol_traded_today']))
        print(vol_data)


def onerror(message):
    print("Error:", message)

def onclose(message):
    print("Connection closed:", message)

def onopen():
    data_type = "SymbolUpdate"

    # Subscribe to the specified symbols and data type
    symbols = ['MCX:CRUDEOIL25SEPFUT']
    fyers.subscribe(symbols=symbols, data_type=data_type)

    # Keep the socket running to receive real-time data
    fyers.keep_running()

def generate_csv(csv_name, field_names, dict):

    if not os.path.isfile(f'{csv_name}.csv'):
        with open(f'{csv_name}.csv', 'a', newline='') as obj:
            dictwriter_object = DictWriter(obj, fieldnames=field_names)
            dictwriter_object.writeheader()
            dictwriter_object.writerow(dict)
            obj.close()
    else:
        with open(f'{csv_name}.csv', 'a', newline='') as obj:
            dictwriter_object = DictWriter(obj, fieldnames=field_names)
            dictwriter_object.writerow(dict)
            obj.close()

# Create a FyersDataSocket instance with the provided parameters
fyers = data_ws.FyersDataSocket(
    access_token=access_token,       # Access token in the format "appid:accesstoken"
    log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=False,                  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,              # Save response in a log file instead of printing it.
    reconnect=True,                  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,               # Callback function to subscribe to data upon connection.
    on_close=onclose,                # Callback function to handle WebSocket connection close events.
    on_error=onerror,                # Callback function to handle WebSocket errors.
    on_message=onmessage             # Callback function to handle incoming messages from the WebSocket.
)

# Establish a connection to the Fyers WebSocket

def main():
    print("Inside main()")
    fyers.connect()

main()

Inside main()
Response {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
Error: KeyError: The key 'exch_feed_time' is missing in the response.
Response {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
Error: KeyError: The key 'exch_feed_time' is missing in the response.


Response {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Error: KeyError: The key 'exch_feed_time' is missing in the response.
Response {'ltp': 5572.0, 'vol_traded_today': 9306, 'last_traded_time': 1756395416, 'exch_feed_time': 1756395422, 'bid_size': 6, 'ask_size': 28, 'bid_price': 5573.0, 'ask_price': 5575.0, 'last_traded_qty': 2, 'tot_buy_qty': 614, 'tot_sell_qty': 904, 'avg_trade_price': 5597.18, 'low_price': 5568.0, 'high_price': 5631.0, 'lower_ckt': 0, 'upper_ckt': 0, 'open_price': 5602.0, 'prev_close_price': 5618.0, 'type': 'sf', 'symbol': 'MCX:CRUDEOIL25SEPFUT', 'ch': -46.0, 'chp': -0.8188}
message exch_feed_time: 2025-08-28 21:07:02
Response {'ltp': 5568.0, 'vol_traded_today': 9348, 'last_traded_time': 1756395423, 'exch_feed_time': 1756395423, 'bid_size': 15, 'ask_size': 1, 'bid_price': 5568.0, 'ask_price': 5569.0, 'last_traded_qty': 10, 'tot_buy_qty': 577, 'tot_sell_qty': 919, 'avg_trade_price': 5597.05, 'low_price': 5568.0, 'high_price': 5631.0, 'lower_ckt':

In [1]:
from fyers_apiv3.FyersWebsocket import data_ws

client_id = open("../client_ID.txt",'r').read()
access_token = open("../access_token.txt",'r').read()

def onmessage(message):
    """
    Callback function to handle incoming messages from the FyersDataSocket Websocket
    Parameters:
            message (dict): The received message from the WebSocket 
    """
    print("Response:", message)
    print("Symbol:",message['symbol'],  "Volume: ",message['vol_traded_today'])


def onerror(message):
    """
    Callback function to handle Websocket errors
    Parameters:
            message (dict): The error message received from the WebSocket 
    """

    print("Error", message)


def onclose(message):
    """
    Callback function to handle Websocket connection close events. 
    """

    print("Connection Closed", message)


def onopen():
    """
    Callback function to subscribe to data type and symbols upon Websocket connection. 
    """
    # Specify the data type and symbols you want to subscribe to
    data_type = "SymbolUpdate"

    # Subscribe to the specified symbols and data types
    symbols = ['MCX:CRUDEOIL25SEPFUT']
    fyers.subscribe(symbols=symbols, data_type=data_type)

    # Keep the socket running to receive real-time data
    fyers.keep_running()


# Create a FyersDataSocket instance with the provided parameters
fyers = data_ws.FyersDataSocket(
    access_token=access_token,      # Access token in format "appid:accesstoken"
    log_path="",                    # Path to save logs. Leave empty to auto-create logs in the current directory
    litemode=False,                 # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,            # Save response in a log file instead of printing it.
    reconnect=True,                 # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,              # Callback function to subscribe to data upon connection.
    on_close=onclose,               # Callback function to handle WebSocket connection close events.
    on_error=onerror,               # Callback function to handle WebSocket errors.
    on_message=onmessage            # Callback function to handle incoming messages from the WebSocket.
)

fyers.connect()

Response: {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
Error KeyError: The key 'symbol' is missing in the response.
Response: {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
Error KeyError: The key 'symbol' is missing in the response.


Response: {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Error KeyError: The key 'symbol' is missing in the response.
Response: {'ltp': 5570.0, 'vol_traded_today': 7227, 'last_traded_time': 1756390520, 'exch_feed_time': 1756390520, 'bid_size': 15, 'ask_size': 7, 'bid_price': 5569.0, 'ask_price': 5570.0, 'last_traded_qty': 1, 'tot_buy_qty': 528, 'tot_sell_qty': 966, 'avg_trade_price': 5600.0, 'low_price': 5569.0, 'high_price': 5631.0, 'lower_ckt': 0, 'upper_ckt': 0, 'open_price': 5602.0, 'prev_close_price': 5618.0, 'type': 'sf', 'symbol': 'MCX:CRUDEOIL25SEPFUT', 'ch': -48.0, 'chp': -0.8544}
Symbol: MCX:CRUDEOIL25SEPFUT Volume:  7227
Response: {'ltp': 5572.0, 'vol_traded_today': 7236, 'last_traded_time': 1756390521, 'exch_feed_time': 1756390522, 'bid_size': 36, 'ask_size': 23, 'bid_price': 5570.0, 'ask_price': 5573.0, 'last_traded_qty': 1, 'tot_buy_qty': 562, 'tot_sell_qty': 956, 'avg_trade_price': 5599.96, 'low_price': 5569.0, 'high_price': 5631.0, 'lower_ckt': 0, 'upp